In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/final_project'

In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications import MobileNetV2, ResNet50
from keras.applications.vgg16 import VGG16

import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix

from PIL import Image


In [ ]:
# List of emotions on the dataset we want to include (using comments -#- to exlcude).

EMOTIONS=[
    'fear',
    'disgust',
    'neutral',
    'happy',
    'angry'
]

In [ ]:
# Load dataset images into the DataFrame.

dataset = []

for emotion in EMOTIONS:
    print(emotion)
    if os.path.isdir(f'dataset/{emotion}'):
        for image_filename in os.listdir(f'dataset/{emotion}'):
            image = Image.open(f'dataset/{emotion}/{image_filename}')
            pixels = np.array(image).astype(float)
            dataset.append((pixels, emotion))

df = pd.DataFrame(dataset, columns=['pixels', 'emotion'])

In [ ]:
df.head()

In [ ]:
df['emotion'].unique()

In [ ]:
# Normalization of the data

X = np.array([MinMaxScaler().fit_transform(e) for e in df['pixels']])
Y = LabelBinarizer().fit_transform(df['emotion'])

In [ ]:
X[0].shape

In [ ]:
# Split dataset into train and test data.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
# Attributes for input (images width, high and channel) and output (number of labels).

_, IMAGE_WIDTH, IMAGE_HEIGHT = X.shape
_, NUM_LABELS = Y.shape
NUM_CHANNELS = 1

In [ ]:
# Create a Convolutional model.

modelConv = Sequential([
    Conv2D(32, (3,3), padding="Same", activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS)),
    BatchNormalization(),
    Conv2D(32, (5,5), padding="Same", activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.5),
    Conv2D(64, (3,3), padding="Same", activation='relu'),
    BatchNormalization(),
    Conv2D(64, (5,5), padding="Same", activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.5),
    Conv2D(128, (3,3), padding="Same", activation='relu'),
    BatchNormalization(),
    Conv2D(128, (5,5), padding="Same", activation='relu'),
    MaxPooling2D((2,2)),
    Dropout(0.5),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(NUM_LABELS, activation='softmax'),
])

modelConv.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
# Train the convolutional model.

historyConv = modelConv.fit(X_train, Y_train, 
                            batch_size=32, epochs=25, 
                            validation_data=(X_test, Y_test), 
                            callbacks=[TensorBoard(log_dir="logs/modelConv")])

In [ ]:
score, acc = modelConv.evaluate(X_test, Y_test, batch_size=100)
print('Test score:', score)
print("Test accuracy:", acc)

In [ ]:
# Load dataset images into the DataFrame and convert into RGB (required by MobileNetV2)

dataset = []

for emotion in EMOTIONS:
    print(emotion)
    if os.path.isdir(f'dataset/{emotion}'):
        for image_filename in os.listdir(f'dataset/{emotion}'):
          image = Image.open(f'dataset/{emotion}/{image_filename}').convert("RGB")
          pixels = np.array(image).astype(float)
          dataset.append((pixels, emotion))

df = pd.DataFrame(dataset, columns=['pixels', 'emotion'])

In [ ]:
# Normalizacion of the data.

X = np.array((df['pixels']/255).values.tolist())
Y = LabelBinarizer().fit_transform(df['emotion'])

In [ ]:
# Split dataset into train and test data.

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Attributes for input (images width, high and channel) and output (number of labels).

_, IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS = X.shape
_, NUM_LABELS = Y.shape

In [ ]:
base_model = MobileNetV2(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS),
                         include_top=False,
                         weights='imagenet')


modelMobNet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(NUM_LABELS, activation='softmax')
])

modelMobNet.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [ ]:
# Train the MobileNetV2 based model.

historyMobNet = modelMobNet.fit(X_train, Y_train,
                                batch_size=32, epochs=25, 
                                validation_data=(X_test, Y_test),
                                callbacks=[TensorBoard(log_dir="logs/modelMobNet")])


In [ ]:
score, acc = modelMobNet.evaluate(X_test, Y_test, batch_size=100)
print('Test score:', score)
print("Test accuracy:", acc)

In [ ]:
base_model = ResNet50(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS),
                      include_top=False, 
                      weights='imagenet')

base_model.trainable = False

modelResNet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(NUM_LABELS, activation='softmax')
])

modelResNet.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])


In [ ]:
# Train the ResNet50 based model.

historyResNet = modelResNet.fit(X_train, Y_train,
                                batch_size=32, epochs=25, 
                                validation_data=(X_test, Y_test),
                                callbacks=[TensorBoard(log_dir="logs/modelResNet")])

In [ ]:
score, acc = modelResNet.evaluate(X_test, Y_test, batch_size=100)
print('Test score:', score)
print("Test accuracy:", acc)

In [ ]:
# Create a VGG model.
base_model = VGG16(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, NUM_CHANNELS),
                      include_top=False, 
                      weights='imagenet')

base_model.trainable = False

modelVGG = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(NUM_LABELS, activation="softmax")
])

# Compilar el modelo
modelVGG.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the MobileVGG based model.

historyVGG = modelVGG.fit(X_train, Y_train,
                                batch_size=32, epochs=25, 
                                validation_data=(X_test, Y_test),
                                callbacks=[TensorBoard(log_dir="logs/modelVGG")])

In [ ]:
score, acc = modelVGG.evaluate(X_test, Y_test, batch_size=100)
print('Test score:', score)
print("Test accuracy:", acc)

In [ ]:
# Show lost and accuracy for train and test for the convolutional network.

fig, axs = plt.subplots(1, 2, figsize=(20, 6))
axs[0].plot(historyConv.history['loss'])
axs[0].plot(historyConv.history['val_loss'])
axs[1].plot(historyConv.history['accuracy'])
axs[1].plot(historyConv.history['val_accuracy'])

In [ ]:
# Show lost and accuracy for train and test for the MobileNetv2 based network.

fig, axs = plt.subplots(1, 2, figsize=(20, 6))
axs[0].plot(historyMobNet.history['loss'])
axs[0].plot(historyMobNet.history['val_loss'])
axs[1].plot(historyMobNet.history['accuracy'])
axs[1].plot(historyMobNet.history['val_accuracy'])

In [ ]:
# Show lost and accuracy for train and test for the ResNet50 based network.

fig, axs = plt.subplots(1, 2, figsize=(20, 6))
axs[0].plot(historyResNet.history['loss'])
axs[0].plot(historyResNet.history['val_loss'])
axs[1].plot(historyResNet.history['accuracy'])
axs[1].plot(historyResNet.history['val_accuracy'])

In [ ]:
# Show lost and accuracy for train and test for the ResNet50 based network.

fig, axs = plt.subplots(1, 2, figsize=(20, 6))
axs[0].plot(historyVGG.history['loss'])
axs[0].plot(historyVGG.history['val_loss'])
axs[1].plot(historyVGG.history['accuracy'])
axs[1].plot(historyVGG.history['val_accuracy'])

In [ ]:
# Save trained models

modelConv.save('models/modelConv_cleaned.h5')
modelMobNet.save('models/modelMobNet_cleaned.h5')
modelResNet.save('models/modelResNet_cleaned.h5')
modelVGG.save('models/modelVGG_cleaned.h5')
